In [1]:
from train.train_model import Train

train = Train()
train.start_training()

/home/vozman/projects/slides/slide-analysis-nn/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


GPU not found


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [4]:
from prediction.predict import Predict

predict_example = Predict()
prediction = predict_example.predict_slide('/home/vozman/projects/slides/slide-analysis-nn/train/datasets/source/slide_images/Tumor_011.tif', area_to_predict=((52000, 140000), (75000, 160000)))
prediction.save_as_asap_annotations(truth_xml_path='/home/vozman/projects/slides/slide-analysis-nn/train/datasets/source/slide_images/Tumor_011_true.xml')
print(prediction)

predict
0:217


[1523206305.2440455, 1523206305.2440455, 1523206305.2440455, 1523206305.2440455, 0.0036385059356689453]
est = 3060516 days, 8:56:30.524200
1:217


[1523206305.2440455, 1523206305.2440455, 1523206305.2440455, 0.0036385059356689453, 0.0001876354217529297]
est = 2284809 days, 10:59:19.793610
2:217


[1523206305.2440455, 1523206305.2440455, 0.0036385059356689453, 0.0001876354217529297, 0.00022864341735839844]
est = 1516154 days, 10:10:51.162308
3:217


[1523206305.2440455, 0.0036385059356689453, 0.0001876354217529297, 0.00022864341735839844, 0.00023865699768066406]
est = 754551 days, 6:31:04.628906
4:217


...

[0.013162851333618164, 0.0002560615539550781, 0.0002765655517578125, 2.5388436317443848, 4.391358137130737]
est = 0:04:41.922228


15:217


[0.0002560615539550781, 0.0002765655517578125, 2.5388436317443848, 4.391358137130737, 4.602684497833252]
est = 0:07:45.950123


Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt: 


KeyboardInterrupt: 